In [3]:
import pandas as pd
import json
import os

# ==========================================
# FILE PATH (SESUAI GAMBAR ANDA)
# ==========================================
nama_file = r"D:\UNSRI_DATA\_SKRIPSI\PROGRAM\v1\pipeline_skripsi\features_v6.4\probe_features_v6.4.json"

if not os.path.isfile(nama_file):
    print(f"ERROR: File tidak ditemukan di: {nama_file}")
else:
    print(f"Membaca file: {nama_file}...")
    with open(nama_file, 'r') as f:
        data = json.load(f)

    df = pd.json_normalize(data)

    # ==========================================
    # PEMBERSIHAN DATA (PENTING!)
    # ==========================================
    target_cols = ['brisque_original', 'brisque_restored', 'niqe_original', 'niqe_restored']
    
    # 1. Konversi ke numerik
    for col in target_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 2. CLIPPING: Ubah nilai negatif menjadi 0
    # Ini standar agar skor 'minus' dianggap sebagai kualitas sempurna (0)
    print("Melakukan normalisasi skor (nilai minus diubah ke 0)...")
    df['brisque_restored'] = df['brisque_restored'].apply(lambda x: 0 if x < 0 else x)
    df['brisque_original'] = df['brisque_original'].apply(lambda x: 0 if x < 0 else x)

    # ==========================================
    # DEFINISI KATEGORI
    # ==========================================
    def get_kategori_jarak(d):
        if pd.isna(d): return "Unknown"
        if d < 7: return "1. Jarak Dekat (<7m)"
        elif 7 <= d <= 12: return "2. Jarak Menengah (7-12m)"
        else: return "3. Jarak Jauh (>12m)"

    def get_kategori_tinggi(h):
        if pd.isna(h): return "Unknown"
        h = round(h, 1)
        if h == 1.5: return "1. Ketinggian 1.5m"
        elif h == 3.0: return "2. Ketinggian 3m"
        elif h == 4.0: return "3. Ketinggian 4m"
        elif h == 5.0: return "4. Ketinggian 5m"
        else: return f"Lainnya ({h}m)"

    if 'metadata.distance_m' in df.columns:
        df['Kategori_Jarak'] = df['metadata.distance_m'].apply(get_kategori_jarak)
        df['Kategori_Tinggi'] = df['metadata.height_m'].apply(get_kategori_tinggi)

        # HITUNG RATA-RATA
        hasil_jarak = df.groupby('Kategori_Jarak')[target_cols].mean().reset_index()
        hasil_tinggi = df.groupby('Kategori_Tinggi')[target_cols].mean().reset_index()

        # FORMAT OUTPUT
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        pd.set_option('display.float_format', '{:.2f}'.format) # 2 angka belakang koma cukup

        print("\n" + "="*60)
        print("HASIL ANALISIS PENGARUH JARAK (Rata-rata Skor)")
        print("   *Nilai semakin kecil = Kualitas semakin bagus")
        print("="*60)
        print(hasil_jarak.to_string(index=False))

        print("\n" + "="*60)
        print("HASIL ANALISIS PENGARUH KETINGGIAN (Rata-rata Skor)")
        print("="*60)
        print(hasil_tinggi.to_string(index=False))
        
    else:
        print("Error: Metadata kolom tidak ditemukan.")

Membaca file: D:\UNSRI_DATA\_SKRIPSI\PROGRAM\v1\pipeline_skripsi\features_v6.4\probe_features_v6.4.json...
Melakukan normalisasi skor (nilai minus diubah ke 0)...

HASIL ANALISIS PENGARUH JARAK (Rata-rata Skor)
   *Nilai semakin kecil = Kualitas semakin bagus
           Kategori_Jarak  brisque_original  brisque_restored  niqe_original  niqe_restored
     1. Jarak Dekat (<7m)             30.96              2.96           7.81           4.60
2. Jarak Menengah (7-12m)             43.08              1.12            NaN           4.56
     3. Jarak Jauh (>12m)             57.17              1.81            NaN           4.35

HASIL ANALISIS PENGARUH KETINGGIAN (Rata-rata Skor)
   Kategori_Tinggi  brisque_original  brisque_restored  niqe_original  niqe_restored
1. Ketinggian 1.5m             41.48              4.44           7.82           4.89
  2. Ketinggian 3m             41.68              1.76           7.96           4.73
  3. Ketinggian 4m             41.74              0.83          